# Merging EB terms-  NLS -  Encyclopaedia Britannica


### Loading the necessary libraries

In [4]:
import yaml
import matplotlib.pyplot as plt
import numpy as np
import collections
import matplotlib as mpl

In [25]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize

### Functions

In [6]:
def read_query_results(filename):
    with open('./results_NLS/'+filename, 'r') as f:
        query_results = safe_load(f)
    return query_results


In [7]:
def write_query_results(filename, results):
    with open('./results_NLS/'+filename, 'w') as f:
        documents = yaml.dump(results, f)

In [31]:
def create_dataframe(data):
    return pd.DataFrame(data.items())

In [9]:
def merge_terms(query_results):
    for page in query_results:
        for element in query_results[page]:
            if "previous_page" in element['term']:
                current_page=int(element['text_unit_id'].split("Page")[1])
                current_definition= element["definition"]
                previous_page_number= current_page-1
                previous_page="Page"+str(previous_page_number)
                num_article_words=element["num_article_words"]
                try:
                    if query_results[previous_page]:
                        flag_prev = 1
                except:
                    flag_prev = 0
                while not flag_prev:
                    previous_page_number = previous_page_number -1
                    previous_page="Page"+str(previous_page_number)
                    try: 
                        if query_results[previous_page]:
                            flag_prev = 1
                    except:
                        flag_prev = 0
        
                
                for prev_elements in query_results[previous_page]:
                    if prev_elements["last_term_in_page"]:
                        prev_elements["definition"]+=current_definition
                        prev_elements["num_article_words"]+=num_article_words
                    
                    prev_elements["num_page_words"]+=num_article_words
                    
                for update_element in query_results[page]:
                    update_element["num_page_words"]-=num_article_words
                    update_element["num_articles"]-=1
                    
                element_index=query_results[page].index(element)
                del query_results[page][element_index]
    return query_results

In [10]:
query_results=read_query_results('results_eb_first')
query_results_updated=merge_terms(query_results)

In [11]:
write_query_results("results_eb_first_updated", query_results_updated)

In [32]:
df=create_dataframe(query_results_updated)

In [36]:
df[0]

0      Page100
1      Page101
2      Page102
3      Page103
4      Page104
        ...   
715     Page93
716     Page94
717     Page95
718     Page96
719     Page99
Name: 0, Length: 720, dtype: object

In [38]:
df.iloc[0][1]

[{'archive_filename': '/Users/rosafilgueira/HW-Work/NLS-Fellowship/work/defoe/nls-data-encyclopaediaBritannica/first_edition/144133901/',
  'definition': "with a plough that has no mold-board, for the horse to go in that draws the drill, direfting himself with his eye by the flakes. 8. The flowing should be fmifhed about the end of September, or beginning of October. 9. The furrows must be traced the long way of the land, that as little ground as possible may be lost in headlands i 10. The rows, if it can be done, ftiould run down the' slope of the land, that the water may get the caller off. 11. The feed-wheat must be plunged into a tub of lime-water, andftirred, that the light corn may come to the surface and be Ikimmcd off. 12. The feed must next be flpread on a floor, and frequently stirred, till it is dry enough to run through the valves of the happer of the drill. 13. To prevent fmut, the feed may be put into a lye of alhes and lime. 14. After the happers of the drill are filled,